In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df=pd.read_csv('shopping_trends.csv')

In [4]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


In [5]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.749949,25.351538
std,1125.977353,15.207589,23.685392,0.716223,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.700000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [74]:
df['Color'].unique()

array(['Gray', 'Maroon', 'Turquoise', 'White', 'Charcoal', 'Silver',
       'Pink', 'Purple', 'Olive', 'Gold', 'Violet', 'Teal', 'Lavender',
       'Black', 'Green', 'Peach', 'Red', 'Cyan', 'Brown', 'Beige',
       'Orange', 'Indigo', 'Yellow', 'Magenta', 'Blue'], dtype=object)

# Задание 1.1

Гипотезы
1. Покупатели с подписками имеют большую частоту покупок
2. Покупатели с промокодами или скидками готовы покупать менее рейтинговые товары
3. Покупатели в северных штатах зимой предпочитают доставку
4. Молодые покупатели предпочитают яркие цвета
5. Пожилые люди чаще предпочитают оплату наличными

In [59]:
from scipy.stats import ttest_ind

frequency_mapping = {
    'Weekly': 52,       # 52 раза в год
    'Bi-Weekly': 26,    # 26 раз в год
    'Fortnightly': 26,  # аналогично Bi-Weekly?
    'Monthly': 12,      # 12 раз в год
    'Quarterly': 4,     # 4 раза в год
    'Every 3 Months': 4, # аналогично Quarterly?
    'Annually': 1       # 1 раз в год
}
df['FrequencyNumeric'] = df['Frequency of Purchases'].map(frequency_mapping)

subscription_group = df[df['Subscription Status'] == 'Yes']['FrequencyNumeric']
non_subscription_group = df[df['Subscription Status'] == 'No']['FrequencyNumeric']

print("Средняя частота покупок с подпиской:", subscription_group.mean())
print("Средняя частота покупок без подписки:", non_subscription_group.mean())

t_stat, t_pvalue = ttest_ind(subscription_group, non_subscription_group)
print("Результаты t-теста:", t_stat, "p-значение:", t_pvalue)

if t_pvalue < 0.05:
    print("Гипотеза о том, что покупатели с подпиской имеют более высокую частоту покупок, подтверждается статистически значимо.")
else:
    print("Нет статистически значимых доказательств того, что покупатели с подпиской имеют большую частоту покупок.")


Средняя частота покупок с подпиской: 17.954415954415953
Средняя частота покупок без подписки: 17.292588689848962
Результаты t-теста: 1.0916222533432596 p-значение: 0.27506663738558235
Нет статистически значимых доказательств того, что покупатели с подпиской имеют большую частоту покупок.


In [66]:
promo_df=df[(df['Discount Applied'] == 'Yes') | (df['Promo Code Used'] == 'Yes')]
non_promo_df=df[(df['Discount Applied'] == 'No') | (df['Promo Code Used'] == 'No')]
print("Средний рейтинг без промокодов:", non_promo_df['Review Rating'].mean())
print("Средний рейтинг с промокодами:", promo_df['Review Rating'].mean())

Средний рейтинг без промокодов: 3.7577147998200626
Средний рейтинг с промокодами: 3.739654144305307


Нет зависимости

In [90]:
northern_states = ['Maine', 'Massachusetts', 'Rhode Island', 'Oregon', 'Wyoming', 
                   'Montana', 'New Hampshire', 'New York', 'North Dakota', 'Illinois', 
                   'Indiana', 'Ohio', 'New Jersey', 'Vermont', 'Idaho', 'Pennsylvania', 
                   'Connecticut', 'South Dakota', 'Minnesota', 'Washington', 
                   'Wisconsin', 'Michigan', 'Alaska']

winter_northern_df = df[(df['Location'].isin(northern_states)) & (df['Season'] == 'Winter')]

# для северных штатов, 1 для доставки, 0 для самовывоза
winter_northern_delivery = winter_northern_df['Shipping Type'].apply(lambda x: 1 if x != 'Store Pickup' else 0)

# для всей выборки: 1 для доставки, 0 для самовывоза
total_delivery = df['Shipping Type'].apply(lambda x: 1 if x != 'Store Pickup' else 0)

delivery_ratio_north = winter_northern_delivery.mean()
delivery_ratio_total = total_delivery.mean()

print(f"Доля доставок для северных штатов зимой: {delivery_ratio_north:.2%}")
print(f"Доля доставок для всей выборки: {delivery_ratio_total:.2%}")

t_stat, p_value = ttest_ind(winter_northern_delivery, total_delivery, equal_var=False)

print("Результаты t-теста:")
print(f"t-статистика: {t_stat}")
print(f"p-значение: {p_value}")


Доля доставок для северных штатов зимой: 83.98%
Доля доставок для всей выборки: 83.33%
Результаты t-теста:
t-статистика: 0.3588629011963514
p-значение: 0.7198283818785064


Нет зависимости

In [76]:
bright_colors = ['Turquoise', 'Pink', 'Purple', 'Gold', 'Violet', 'Teal', 
                 'Green', 'Red', 'Cyan', 'Orange', 'Yellow', 'Magenta', 'Blue']

df['IsYoung'] = df['Age'] < 30

bright_purchases = df[df['Color'].isin(bright_colors)]

# Доля покупок ярких цветов для молодых покупателей
young_bright_ratio = bright_purchases[bright_purchases['IsYoung']].shape[0] / df[df['IsYoung']].shape[0]

#Для других покупателей
non_young_bright_ratio = bright_purchases[~bright_purchases['IsYoung']].shape[0] / df[~df['IsYoung']].shape[0]

print(f"Доля покупок ярких цветов среди молодых покупателей: {young_bright_ratio:.2%}")
print(f"Доля покупок ярких цветов среди других покупателей: {non_young_bright_ratio:.2%}")

Доля покупок ярких цветов среди молодых покупателей: 53.08%
Доля покупок ярких цветов среди других покупателей: 52.08%


Нет зависимости

In [89]:
#Пожилые люди чаще предпочитают оплату наличными
df['IsOld'] = df['Age'] > 65

cash_purchases = df[df['Preferred Payment Method']=='Cash']

old_cash_ratio = cash_purchases[cash_purchases['IsOld']].shape[0] / df[df['IsOld']].shape[0]

#Для других покупателей
non_old_cash_ratio = cash_purchases[~cash_purchases['IsOld']].shape[0] / df[~df['IsOld']].shape[0]

print(f"Доля покупок наличными среди пожилых покупателей: {old_cash_ratio:.2%}")
print(f"Доля покупок наличными среди других покупателей: {non_old_cash_ratio:.2%}")

Доля покупок наличными среди пожилых покупателей: 17.18%
Доля покупок наличными среди других покупателей: 17.18%


Нет зависимости

Из за того что данные синтетические ни одна из гипотез не подтвердилась :(

# Задание 1.2
Самый популярный товар

In [39]:
modes = df['Item Purchased'].mode()
print("Самые популярные товары:")
for value in modes:
    print(value)

Самые популярные товары:
Blouse
Jewelry
Pants


Распределение покупателей по полу

In [7]:
gender_counts = df['Gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']  # Переименовываем столбцы для удобства

# Построение графика
fig = px.bar(gender_counts, x='Gender', y='Count', 
             title='Распределение покупателей по полу', 
             labels={'Gender': 'Пол', 'Count': 'Количество покупателей'})

fig.show()

Какой пол (и отдельно возраст) покупает больше всего, чаще всего, самые дорогие товары

In [8]:
#Пол покупающий больше всего
df.groupby('Gender')['Purchase Amount (USD)'].sum().idxmax()

'Male'

In [9]:
#Возраст покупающий больше всего
df.groupby('Age')['Purchase Amount (USD)'].sum().idxmax()

49

In [10]:
#Пол покупающий чаще
df['Gender'].mode()[0]

'Male'

In [11]:
#Возраст покупающий чаще
df['Age'].mode()[0]

69

In [12]:
#Пол покупающий самые дорогие товары
df.loc[df['Purchase Amount (USD)'].idxmax(), 'Gender']

'Male'

In [13]:
#Возраст покупающий самые дорогие товары
df.loc[df['Purchase Amount (USD)'].idxmax(), 'Age']

20

Есть ли зависимость между цветом одежды и сезоном

In [50]:
import pandas as pd
import plotly.express as px

color_season_table = pd.crosstab( df['Season'], df['Color'])

fig = px.imshow(color_season_table, text_auto=True, color_continuous_scale='ylgnbu', 
                title='Распределение цвета одежды по сезонам')

fig.update_layout(
    xaxis_title='Цвет одежды',
    yaxis_title='Сезон'
)
fig.show()

In [51]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(color_season_table)

print("Значение хи-квадрат:", chi2)
print("p-значение:", p)

Значение хи-квадрат: 64.6506366841786
p-значение: 0.718551112120402


p>0.05, значит нет зависимость между цветом одежды и сезоном

Cезонный mau (уникальных пользователей за сезон) и его динамика

In [33]:
import plotly.express as px

seasonal_counts = df.groupby('Season').size().reset_index(name='Count')

season_order = ['Winter', 'Spring', 'Summer', 'Fall']
seasonal_counts['Season'] = pd.Categorical(seasonal_counts['Season'], categories=season_order, ordered=True)
seasonal_counts = seasonal_counts.sort_values('Season')

fig = px.line(seasonal_counts, x='Season',y='Count', title='Сезонный MAU')

fig.update_layout(
    xaxis_title='Сезон',
    yaxis_title='Количество'
)

fig.show()

Самая популярная буква в названии одежды

In [69]:
from collections import Counter

clothing_df=df[df['Category']=='Clothing']
all_names = ''.join(clothing_df['Item Purchased'].dropna()).replace(' ', '').lower()
letter_counts = Counter(all_names)
most_common_letter = letter_counts.most_common(1)[0]

print(f"Самая популярная буква в названиях одежды: '{most_common_letter[0]}', встречается {most_common_letter[1]} раз(а).")

Самая популярная буква в названиях одежды: 's', встречается 2068 раз(а).
